# Desafio
Realizado por: Vitor Klein

## Etapa 1:
Leitura dos arquivos csv. 
Transformação para gravação ao banco de dados.

## Etapa 2:
Criação do banco de dados
Criação das tabelas
Carga dos arquivos ao banco

## Etapa 3:
Resposta as perguntas de negocio com SQL



In [ ]:
import pandas as pd
import sqlite3

FILE_PAT_PIB = 'D:\OneDrive - K Data BI\K Data BI\Python_Projects\desafio_eleflow\data\PIB_Cepea.xlsx'
df_pib = pd.read_excel(FILE_PAT_PIB, sheet_name='PIB', header=7, index_col=0)

In [ ]:
df_pib= df_pib[['Ramo Agrícola (A+B+C+D).1', 'Ramo Pecuário (A+B+C+D).1']]
df_pib.reset_index(inplace=True)
df_pib.columns = ['Ano','Agrícola', 'Pecuário']
df_pib = df_pib[df_pib['Ano'].astype(str).str.isdigit()]
df_pib = pd.melt(df_pib, id_vars=['Ano'], var_name='Ramo',value_name='Valor',value_vars=['Agrícola', 'Pecuário'])


In [ ]:
connection = sqlite3.connect('eleflow_database')
cursor = connection.cursor()

In [ ]:
query = """
    CREATE TABLE IF NOT EXISTS pib(
        ano int,
        ramo varchar(30), 
        valor_pib numeric(14,2)) 
"""

query2 = """
    CREATE TABLE IF NOT EXISTS producao(
        ano integer,
        mes varchar(20),
        uf varchar(30),
        valor numeric(14,2)
    )
"""

query3 = """
    CREATE TABLE IF NOT EXISTS area(
        ano integer,
        mes varchar(20),
        uf varchar(30),
        valor numeric(14,2)
    )
"""

In [ ]:
df_pib.to_sql('PIB', con=engine, if_exists='replace')

In [ ]:
FILE_PATH_PRODUCAO = 'D:\OneDrive - K Data BI\K Data BI\Python_Projects\desafio_eleflow\data\produção_2019.xlsx'
df = pd.read_excel(FILE_PATH_PRODUCAO, sheet_name='area prod uf', header=5, index_col=0)

In [ ]:
df.reset_index(inplace=True)
df.columns = ['UF', 'Área(ha)', 'Março1', 'Junho1', 'Produção','Março2','Junho2']

In [ ]:
df_area = df[['UF', 'Área(ha)', 'Março1', 'Junho1']]
df_production = df[['UF', 'Produção', 'Março2', 'Junho2']]
df_area = df_area.rename(columns={'Março1':'Março', 'Junho1':'Junho'})
df_production = df_production.rename(columns={'Março2':'Março', 'Junho2':'Junho'})

In [ ]:
df_production['Ano'] = 2019
df_area['Ano'] = 2019

In [ ]:
df_production = df_production[df_production['Produção'].notna()]
df_area = df_area[df_area['Área(ha)'].notna()]

In [ ]:
df_area = pd.melt(df_area, id_vars=['Ano','UF'], var_name='Mês',value_name='Valor',value_vars=['Março', 'Junho'])
df_production = pd.melt(df_production, id_vars=['Ano','UF'], var_name='Mês',value_name='Valor',value_vars=['Março', 'Junho'])